In [1]:
from pathlib import Path
import scanpy as sc
import re, sys
import pandas as pd

In [2]:
def parse_sam(file):
	result = set()
	with open(file, 'r') as inputFile:
		for line in inputFile.readlines():
			fields = line.split('\t')
			acc = fields[2]
			for item in fields[19:]:
				if item.find('CB:Z:') > -1:
					barcode = re.sub(r'CB:Z:', r'', item)
				if item.find('UB:Z:') > -1:
					umi = re.sub(r'UB:Z:', '', item)
			result.add(f'{acc}\t{barcode}\t{umi}')
	return result

def generate_table(data, target_set, target_name, suffix):
	filtered_data = []
	for item in data:
		if item.split('\t')[0] in target_set:
			filtered_data.append(item)

	with open('exp_aae_denv.txt', 'w') as outputFile:
		for item in filtered_data:
			outputFile.write(item)
	table = pd.read_table('exp_aae_denv.txt', header=None, delimiter='\t')
	table.columns = ['Accession', 'Barcode', 'UMI']
	table['Accession'] = table['Accession'].map(lambda x: target_name[x])
	table = table[table['Barcode'] != '-']
	table = table[table['UMI'] != '-']
	table = table.groupby(['Accession', 'Barcode'])['UMI'].count().reset_index()
	table['Barcode'] = table['Barcode'] + suffix
	Path('exp_aae_denv.txt').unlink()
	return table

def write2csv(adata, table, target_name, suffix):
	umap_df = pd.DataFrame(
	adata.obsm['X_umap'], 
	index=adata.obs_names,
	columns=['UMAP_1', 'UMAP_2']  # 列名可自定义
	)
	umap_df = pd.concat([adata.obs['cluster'], umap_df], axis=1).reset_index()
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2']
	umap_df.to_csv('temp.csv', header=True, index=False)
	umap_df = pd.read_csv('temp.csv', header=0)
	for name in target_name.values():
		temp_column = table[table['Accession'] == name][['Barcode', 'UMI']]
		umap_df = umap_df.merge(temp_column, how='left', on='Barcode').fillna(0)
	umap_df.columns = ['Barcode', 'Cluster', 'UMAP_1', 'UMAP_2'] + list(target_name.values())
	umap_df.to_csv(f'exp_{suffix}.csv', index=False)
	Path('temp.csv').unlink()
	print(umap_df)

In [4]:
data = parse_sam('exp_aae_bl.sam')

In [5]:
target_set = ['MW174761.1', 'ON949933.1']
target_name = {
	'MW174761.1': 'Totichi',
	'ON949933.1': 'HKIFV'
}
table = generate_table(data, target_set, target_name, '-aae-bl')

In [6]:
adata = sc.read('../../../processed_h5ad/aae_bl_denv.h5ad')
adata = adata[adata.obs['batch'] == 'aae-bl']
write2csv(adata, table, target_name, 'aae_bl')

                                    Barcode    Cluster    UMAP_1     UMAP_2  \
0      CCAATGTCT_ACCAGGTCA_AACAAGTGG-aae-bl         EC  2.416772  16.328863   
1      TGTGAGTCA_ACGTTACGA_AACAAGTGG-aae-bl       EE-2 -4.409736   0.744095   
2      CCAATGTCT_ACTCATCGT_AACAAGTGG-aae-bl  EC-like-1  4.893340   8.834125   
3      CGCCAATGA_AGATACGCA_AACAAGTGG-aae-bl   Cardia-2 -0.592695   2.964518   
4      TGGCATGAG_AGCCACATA_AACAAGTGG-aae-bl  EC-like-1  5.122269  11.434782   
...                                     ...        ...       ...        ...   
10546  AGCTCCTTG_TGGACTTGT_TTGTGTACG-aae-bl  EC-like-3  5.981905  12.061672   
10547  CTTGACGTT_TGGACTTGT_TTGTGTACG-aae-bl  EC-like-1  4.655382  11.184032   
10548  AGTTCAGAG_TGGTCAGTT_TTGTGTACG-aae-bl   Cardia-1 -3.829197   3.455635   
10549  CTCAGAACT_TTCCAATCG_TTGTGTACG-aae-bl  EC-like-1  3.024297   9.910492   
10550  AGGAGCAAT_TTGGTGACC_TTGTGTACG-aae-bl   Cardia-2 -0.842723   3.419367   

       Totichi  HKIFV  
0          0.0    2.0  
1  